# Dependencies

In [ ]:
import shutil, os
from pathlib import Path
import pandas as pd
import random
import numpy as np

In [ ]:
DATASET_INPUT_DIR = f"/kaggle/input/master-data/dataset"

In [ ]:
REPO_DIR = f"/kaggle/working/repos"

if not os.path.isdir(REPO_DIR):
    os.mkdir(REPO_DIR)
os.chdir(REPO_DIR)

CUT_DIR = f"{REPO_DIR}/CUT"
WCT2_DIR = f"{REPO_DIR}/WCT2"

if not os.path.isdir(CUT_DIR):
    !git clone https://github.com/taesungp/contrastive-unpaired-translation.git CUT

os.chdir(CUT_DIR)
!pip install -r requirements.txt

os.chdir(REPO_DIR)

if not os.path.isdir(WCT2_DIR):
    !git clone https://github.com/clovaai/WCT2.git

!pip install dominate

# Photorealistic Style Transfer (WCT^2)

## Preparation of Content and Style Images

The following cell can be adjusted to decide the base experiment whose synthetic training data should be transferred. The seed is used for the random selection of style images of the real dataset for domain adaptation.

In [ ]:
SCENARIO = 1 # 1: tools, 2: hammers
EXPERIMENT = '2baseline'
# 2baseline, 3dr_mat, 4dr_bel, 5dr_dis0, 6dr_dis1, 7dr_hin0, 8dr_hin1, 9dr_mix
SEED = 42

In [ ]:
os.chdir(WCT2_DIR)

CONTENT_DIR = f"{DATASET_INPUT_DIR}/scenario{SCENARIO}/{EXPERIMENT}/images/train"
STYLE_DIR = f"{DATASET_INPUT_DIR}/da/style/scenario{SCENARIO}"
    
if not os.path.exists("data"):
    os.mkdir("data")
else:
    shutil.rmtree("data")
    os.mkdir("data")

os.chdir("data")
if not os.path.exists("style"):
    os.mkdir("style")
if not os.path.exists("content"):
    os.mkdir("content")

rng = random.Random(SEED)

style_files = [p for p in Path(STYLE_DIR).iterdir() if p.is_file()]

for path in os.listdir(CONTENT_DIR):
    if os.path.isdir(path):
        continue

    content_file_name = Path(path).name

    style_file_name = rng.choice(style_files).name

    shutil.copyfile(f"{CONTENT_DIR}/{content_file_name}", f"{WCT2_DIR}/data/content/{content_file_name}")
    shutil.copyfile(f"{STYLE_DIR}/{style_file_name}", f"{WCT2_DIR}/data/style/{content_file_name}")

## Transfer Images with WCT^2

In [ ]:
os.chdir(WCT2_DIR)

OUTPUT_DIR = f"{WCT2_DIR}/results"

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

!python transfer.py --option_unpool cat5 -e -s \
  --content ./data/content --style ./data/style --output ./results/ \
  --image_size 640 --alpha 0.5

## Process Results and Save as ZIP

Transferred images need to be renamed as WCT^2 appends the file name by some text, but we need the images to have the same name as before so that they match the labels for training of YOLO. Processed results are saved to ```/kaggle/working/repos/WCT2/processed/{SCENARIO}_{EXPERIMENT}```. ZIP is saved to ```/kaggle/working/zip``` under file name ```{SCENARIO}_{EXPERIMENT}.zip```.

In [ ]:
os.chdir(WCT2_DIR)

if not os.path.exists("processed"):
    os.mkdir("processed")

if not os.path.exists(f"processed/scenario{SCENARIO}_{EXPERIMENT}"):
    os.mkdir(f"processed/scenario{SCENARIO}_{EXPERIMENT}")
    
for path in os.listdir(OUTPUT_DIR):
    if os.path.isdir(path):
        continue

    file_name = Path(path).stem
    new_name = file_name.split('_')[0]

    shutil.copyfile(f"{WCT2_DIR}/results/{file_name}.jpg", f"{WCT2_DIR}/processed/scenario{SCENARIO}_{EXPERIMENT}/{new_name}.jpg")

os.chdir("/kaggle/working/")
if not os.path.exists("zip"):
    os.mkdir("zip")
    
shutil.make_archive(f"/kaggle/working/zip/wct2_{SCENARIO}_{EXPERIMENT}","zip",f"{WCT2_DIR}/processed/scenario{SCENARIO}_{EXPERIMENT}")

# Image-to-Image-Translation (FastCUT)

## Preparation of Data

The training and test data must be in one shared folder, each dataset divided by the two domains. The test data are the images which will be translated. Subfolders for division must be named ```trainA``` and ```trainB``` (where domain A gets translated to domain B) and ```testA``` & ```testB``` respectively.

In this case, domain A corresponds to the synthetic domain and domain B to the real domain. As described in the thesis, for both training and test data, only the synthetic training data of the experiment ```DR-Hin1``` will be used. For the test data, the ```SCENARIO``` whose DR-Hin1 test data should be translated must be specified.

In [ ]:
os.chdir(CUT_DIR)
os.chdir("datasets")

if not os.path.isdir("images"):
    os.mkdir("images")
else:
    shutil.rmtree("images")
    os.mkdir("images")

DA_DATASET_DIR = f"{DATASET_INPUT_DIR}/da"

In [ ]:
# TRAINING DATA

os.chdir(f"{CUT_DIR}/datasets/images")

# SYNTHETIC IMAGES of DR-Hin1 from both scenarios (domain A)
os.mkdir("trainA")

for i in range(2):
    CONTENT_PATH = f"{DATASET_INPUT_DIR}/scenario{i+1}/8dr_hin1/images/train"
    content_files = [p for p in Path(CONTENT_PATH).iterdir() if p.is_file()]
    sampled_files = np.random.choice(content_files, size=200, replace=False)
    for file in sampled_files:
        file_name = file.name
        shutil.copyfile(f"{CONTENT_PATH}/{file_name}", f"{CUT_DIR}/datasets/images/trainA/{i+1}_{file_name}")

# REAL IMAGES (domain B)
os.mkdir("trainB")
shutil.copytree(f"{DA_DATASET_DIR}/style/scenario1", f"{CUT_DIR}/datasets/images/trainB", dirs_exist_ok=True)
shutil.copytree(f"{DA_DATASET_DIR}/style/scenario2", f"{CUT_DIR}/datasets/images/trainB", dirs_exist_ok=True)

In [ ]:
# TEST DATA
SCENARIO = 1 # 1: tools, 2: hammers

os.chdir(f"{CUT_DIR}/datasets/images")

if os.path.isdir("testA"):
    os.unlink(f"{CUT_DIR}/datasets/images/testA")
if os.path.isdir("testB"):
    os.unlink(f"{CUT_DIR}/datasets/images/testB")

os.symlink(f"{DATASET_INPUT_DIR}/scenario{SCENARIO}/8dr_hin1/images", f"{CUT_DIR}/datasets/images/testA", target_is_directory=True)
os.symlink(f"{CUT_DIR}/datasets/images/trainB", f"{CUT_DIR}/datasets/images/testB", target_is_directory=True)

## Train FastCUT

In [ ]:
os.chdir(CUT_DIR)

!python train.py \
  --dataroot ./datasets/images \
  --name syn2real_fastcut \
  --CUT_mode FastCUT \
  --n_epochs 75 \
  --n_epochs_decay 25 \
  --batch_size 16 \
  --nce_layers 0,2,4,6 \
  --preprocess scale_shortside_and_crop  --crop_size 160 --load_size 640

## Translate Images with trained FastCUT

In [ ]:
os.chdir(CUT_DIR)

OUTPUT_DIR = f"{CUT_DIR}/results"

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
    
!python test.py --dataroot ./datasets/images \
    --CUT_mode FastCUT \
    --phase test \
    --name syn2real_fastcut \
    --load_size 640 --crop_size 640 \
    --preprocess none \
    --num_test 1000

## Save as ZIP

In [ ]:
os.chdir("/kaggle/working/")
if not os.path.exists("zip"):
    os.mkdir("zip")

shutil.make_archive(f"/kaggle/working/zip/fcut_{SCENARIO}_{EXPERIMENT}","zip",f"{CUT_DIR}/results/syn2real_fastcut/test_latest/images/fake_B/")

# Clean Up

In [ ]:
# clear output/working dir
for name in os.listdir(WORKING_DIR):
    path = os.path.join(WORKING_DIR, name)
    if os.path.isfile(path) or os.path.islink(path):
        os.unlink(path)
    else:
        shutil.rmtree(path)